In [13]:
#importing all the necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
import math
import sys

In [14]:
# Generating an iris dataset for classifying the type of flower based on its features
iris = datasets.load_iris()
X, Y = iris.data, iris.target

In [15]:
# Creating a pandas DataFrame from the generated iris dataset
df = pd.DataFrame(X[:,:],columns=['Sepal_length','Sepal_width','Petal_length','Petal_width'])
df['Y']=Y
df

,Sepal_length,Sepal_width,Petal_length,Petal_width,Y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [16]:
# Shuffling the dataset for developing randomness/disorder in the data
df = df.sample(frac=1)

In [17]:
# Defining the structure of a Node of the Classification Decision Tree
class Node():
  def __init__(self):
    self.value = None
    self.column_name = None
    self.size= 0
    self.left = None
    self.right = None
    self.ans = None


Formula for Entropy:

$E(P) = \Sigma_{i=0}^{n}\;-P_{i}\log_{2}(P_{i})$



In [18]:
# Function to calculate the entropy for a dataset
def entropy(Y):
  if Y.empty:
    return 0
  classes = list(Y.value_counts())
  entropy = 0
  for freq in classes:
    entropy += -(freq/Y.shape[0])*math.log(freq/Y.shape[0])
  return entropy

Formula for Gini Index / Gini Impurity:

$ 1-Σ_{i=0}^{n}\;(P_{i})^2 $


In [19]:
# Function to calculate Gini impurity of a dataset
def gini(Y):
  if Y.empty:
    return 0
  classes = list(Y.value_counts())
  gini_index = 0
  for freq in classes:
    gini_index += (freq/Y.shape[0])**2
  return 1-gini_index

In [20]:
# Function to calculate the disorder in a dataset (Gini,Entropy)
def measureDisorder(df,algorithm_choice="Entropy"):
  Y = df['Y']
  if (algorithm_choice == "Gini"):
    return gini(Y)
  elif (algorithm_choice == "Entropy"):
    return entropy(Y)

Formula for Information Gain :

$IG = E(Parent) - Σ_{k=1}^{no. of divides}\; \frac{frequency(divide_{k})}{total}*E(divide_{k}) $


In [21]:
# Function to calculate Information Gain by spliting a dataset
def calInfoGain(df,column_name,parent_entropy,disorder_algo,min_samples_leaf):
  max_info_gain = -sys.maxsize - 1
  split_point = 0
  size = df.shape[0]
  #Preparing the Dataset for easy comparison of values
  df1= df.sort_values(column_name)
  Y = df1["Y"]

  #Iterating from min_samples_leaf to (size-min_samples_leaf) to insure that we get a subset of size atleast min_samples_leaf.
  for i in range(min_samples_leaf,size-min_samples_leaf):

    #Calculating the impurity/disorder due to each subset of dataset
    impurity1 = measureDisorder(df1.iloc[:i,:],disorder_algo)
    impurity2 = measureDisorder(df1.iloc[i:,:],disorder_algo)

    #Implementing the formula for Information Gain
    #where w1 & w2 are the weights of each impurity values
    w1 = (i+1)/size
    w2 = (size-i-1)/size
    info_gain = parent_entropy - w1*impurity1 - w2*impurity2

    #Updating the value of max_info_gain encountered till current iteration and storing the iteration position
    if(info_gain > max_info_gain):
      split_point = i
      max_info_gain = info_gain
  #returning a tuple containig necessary spliting_criteria
  return (max_info_gain,df1[column_name].iloc[split_point],column_name)

In [22]:
# Function to split the dataset with respect to a threshold value of a particular column
def split(df,criteria):
  column_name = criteria[2]
  threshold = criteria[1]
  df_left = df.loc[
      df[column_name] < threshold
  ]
  df_right = df.loc[
      df[column_name] > threshold
  ]

  # To handle the exceptional cases
  if(df_left.empty):
    df_left = df.loc[
        df[column_name] == threshold
    ]
  elif(df_right.empty):
    df_right = df.loc[
        df[column_name] == threshold
    ]
  else:
    df_left = df.loc[
      df[column_name] <= threshold
  ]
  #returning both the left and right subsets for the Decision Node.
  return df_left, df_right

In [23]:
# Function to generate a Decision Tree for a dataset and return its root node
def createDecisionTree(df,max_depth,disorder_algo,min_sample_split,min_samples_leaf=1,min_info_gain=0,curr_depth=0):

  # If the dataset is empty for any leaf node then it doesnt need to be added to the Decision Tree.
  # For this purpose we will return a void inplace of any leaf node.
  if df.empty:
    return

  # Initialize a Node that can be used in feature for creating either Decision Node or Leaf Node.
  root = Node()
  root.size= df.shape[0]

  # Calculating parents impurity for further use in calculating Information Gain
  parent_impurity=measureDisorder(df,disorder_algo)

  # Termination case 1: If the impurity for parent is 0 , means that for a dataset there is no uncertainty.
  # Meaning the whole dataset comprise of data for a single class.
  # So we will mark this state as a Leaf Node with the class of the datasets first entry.
  if(parent_impurity==0):
      root.ans = df['Y'].iloc[0]
      return root

  # Termination case 2: If the depth of the tree is equal to specified depth, we will stop generating further Child nodes based on the current dataset.
  # To convert this node into Leaf Node we will mark this Node with the maximum occuring class in the dataset.
  if(curr_depth > max_depth):
    root.ans = df['Y'].mode().loc[0]
    return root


  size = df.shape[0]
  # Finding the names of all the features in the dataset.
  all_columns = (list(df.columns))
  all_columns.pop(-1)

  #Finding the best feauture for spliting the datasets
  max_info_gain = -sys.maxsize - 1
  split_criteria = ()
  for column in  all_columns:

    # Termination case 3: Ensures that only the dataset with a min number of samples are split
    if(df.shape[0] < min_sample_split):
      root.ans = df['Y'].mode().loc[0]
      return root

    # Termination case 4: Ensures that the dataset can be divided into two subsets such that both the part have atleast min amount of samples.
    if(df.shape[0] <= 2*min_samples_leaf):
      root.ans = df['Y'].mode().loc[0]
      return root

    #Calculating Information Gain by spliting dataset based on a feature
    col_ig = calInfoGain(df,column,parent_impurity,disorder_algo,min_samples_leaf)
    #Finding the best feature / column for spliting the dataset among all the features
    if(col_ig[0] > max_info_gain):
      split_criteria = col_ig
      max_info_gain = col_ig[0]

  #Checking if the info_gain for a split is greater than min_info_gain allowed
  if(max_info_gain < min_info_gain):
      root.ans = df['Y'].mode().loc[0]
      return root

  #Spliting the current dataset according to the spliting feature and its threshold value
  df_left,df_right = split(df,split_criteria)
  print(split_criteria)
  #Inserting the necessary details for creating a Decision Node
  root.value = split_criteria[1]
  root.column_name = split_criteria[2]

  #Creating the left sub-tree
  temp_depth1= curr_depth
  root.left = createDecisionTree(df_left,max_depth,disorder_algo,min_sample_split,min_samples_leaf,min_info_gain,temp_depth1+1)

  #Creating the right sub-tree
  temp_depth2 = curr_depth
  root.right = createDecisionTree(df_right,max_depth,disorder_algo,min_sample_split,min_samples_leaf,min_info_gain,temp_depth2+1)

  return root

In [24]:
Decision_tree = createDecisionTree(df.iloc[:int(df.shape[0]*.9),:],5,"Gini",2)

(0.3332638467518403, 3.0, 'Petal_length')
(0.03234567901234571, 1.9, 'Petal_length')
(0.4150601024717948, 1.7, 'Petal_width')
(0.08769513314967874, 5.1, 'Petal_length')
(0.04160843893673118, 4.9, 'Sepal_length')
(0.03305785123966941, 5.1, 'Petal_length')
(0.00791383219954659, 3.2, 'Sepal_width')
(0.016243752402922064, 3.2, 'Sepal_width')
(0.2777777777777777, 6.4, 'Sepal_length')


In [25]:
# A level-order traversal of the Decision tree for sake of representation and debugging purposes.
def VisualizeTree(root):
  q = []
  q.append(root)
  q.append(None)
  post_last = None
  while(len(q) != 0):
    front = q[0]
    q.pop(0)
    if(front==None):
      print("\n\n")
      if(len(q)!=0):
        q.append(None)
    else:
      print(front.value,":",front.column_name,":",front.size,end=" ; ")
      if(front.left!=None):
        q.append(front.left)
      if(front.right!=None):
        q.append(front.right)

In [26]:
VisualizeTree(Decision_tree)

3.0 : Petal_length : 135 ; 


1.9 : Petal_length : 45 ; 1.7 : Petal_width : 90 ; 


None : None : 44 ; None : None : 1 ; 5.1 : Petal_length : 48 ; 3.2 : Sepal_width : 42 ; 


4.9 : Sepal_length : 46 ; None : None : 2 ; 3.2 : Sepal_width : 34 ; None : None : 8 ; 


None : None : 2 ; 5.1 : Petal_length : 44 ; None : None : 28 ; 6.4 : Sepal_length : 6 ; 


None : None : 42 ; None : None : 2 ; None : None : 2 ; None : None : 4 ; 




In [27]:
# Method to use the trained Decision Tree ("root") and predict the classes of a dataset ("df")
def predict(root,df):
  y_pred = np.zeros(df.shape[0])
  correct_pred = 0
  for i in range (df.shape[0]):
    row = df.iloc[i]
    temp = root
    while(temp.value!=None):
      if(row[temp.column_name]<=temp.value):
        temp = temp.left
      else:
        temp=temp.right
    y_pred[i] = int(temp.ans)
    if(temp.ans == row['Y']):
      correct_pred += 1
  return(y_pred,(correct_pred/df.shape[0])*100)

In [28]:
Y_pred,accuracy = predict(Decision_tree,df.iloc[int(df.shape[0]*.9):,:])
Y_pred

array([0., 0., 0., 2., 0., 2., 0., 1., 2., 0., 1., 1., 1., 1., 2.])

In [29]:
accuracy

93.33333333333333